# Text Preprocessing

In [3]:
!pip install langdetect

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     -------------------------------------- 981.5/981.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993251 sha256=76dd07d05aebca162dd9961dad0306c361dd93ad306ab62562d9d9c51c2d19cd
  Stored in directory: c:\users\thema\appdata\local\pip\cache\wheels\c1\67\88\e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [2]:
# Import libraries
import pandas as pd
import ast
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
import re

# Download NLTK content
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thema\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thema\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thema\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Load in the dataset
df = pd.read_csv("news_dataset.csv")
df.head()

,Unnamed: 0,text,target,title,date
0,0,I was wondering if anyone out there could enli...,7,rec.autos,2022-08-02 13:48:37.251043
1,17,I recently posted an article asking what kind ...,7,rec.autos,2022-08-02 13:48:37.251043
2,29,\nIt depends on your priorities. A lot of peo...,7,rec.autos,2022-08-02 13:48:37.251043
3,56,an excellent automatic can be found in the sub...,7,rec.autos,2022-08-02 13:48:37.251043
4,64,: Ford and his automobile. I need information...,7,rec.autos,2022-08-02 13:48:37.251043


In [6]:
df["text"]

0        I was wondering if anyone out there could enli...
1        I recently posted an article asking what kind ...
2        \nIt depends on your priorities.  A lot of peo...
3        an excellent automatic can be found in the sub...
4        : Ford and his automobile.  I need information...
                               ...                        
11309    Secrecy in Clipper Chip\n\nThe serial number o...
11310    Hi !\n\nI am interested in the source of FEAL ...
11311    The actual algorithm is classified, however, t...
11312    \n\tThis appears to be generic calling upon th...
11313    \nProbably keep quiet and take it, lest they g...
Name: text, Length: 11314, dtype: object

In [8]:
# Drop duplicates and null values
df_cleaned = df.dropna()
df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)

In [10]:
# Step 1: Convert the text to lower case
df["lowercased"] = df["text"].astype(str).str.lower()

In [12]:
# Step 2: Remove slangs from the text
slang_dict = {
    "tbh": "to be honest",
    "omg": "oh my god",
    "lol": "laugh out loud",
    "idk": "I don't know",
    "brb": "be right back",
    "btw": "by the way",
    "imo": "in my opinion",
    "smh": "shaking my head",
    "fyi": "for your information",
    "np": "no problem",
    "ikr": "I know right",
    "asap": "as soon as possible",
    "bff": "best friend forever",
    "gg": "good game",
    "hmu": "hit me up",
    "rofl": "rolling on the floor laughing",
    "w": "win",
    "im": "i'm"
}

def replace_slang(text):
    escaped_slang_words = []
    for word in slang_dict.keys():
        escaped_word = re.escape(word)
        escaped_slang_words.append(escaped_word)
    slang_pattern = r'\b(' + '|'.join(escaped_slang_words) + r')\b'
    
    def replace_match(match):
        slang_word = match.group(0)
        return slang_dict[slang_word.lower()]
    replaced_text = re.sub(slang_pattern, replace_match, text, flags=re.IGNORECASE)
    return replaced_text

df['slangs_replaced'] = df['lowercased'].apply(replace_slang)

In [13]:
# Step 3: Replace contrations in the text with full clause
contractions_dict = {
    "wasn't": "was not",
    "isn't": "is not",
    "aren't": "are not",
    "weren't": "were not",
    "doesn't": "does not",
    "don't": "do not",
    "didn't": "did not",
    "can't": "cannot",
    "couldn't": "could not",
    "shouldn't": "should not",
    "wouldn't": "would not",
    "won't": "will not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "i'm": "i am",
    "you're": "you are",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "we're": "we are",
    "they're": "they are",
    "i've": "i have",
    "you've": "you have",
    "we've": "we have",
    "they've": "they have",
    "i'd": "i would",
    "you'd": "you would",
    "he'd": "he would",
    "she'd": "she would",
    "we'd": "we would",
    "they'd": "they would",
    "i'll": "i will",
    "you'll": "you will",
    "he'll": "he will",
    "she'll": "she will",
    "we'll": "we will",
    "they'll": "they will",
    "let's": "let us",
    "that's": "that is",
    "who's": "who is",
    "what's": "what is",
    "where's": "where is",
    "when's": "when is",
    "why's": "why is"
}

escaped_contractions = []

for contraction in contractions_dict.keys():
    escaped_contraction = re.escape(contraction)
    escaped_contractions.append(escaped_contraction)

joined_contractions = "|".join(escaped_contractions)
contractions_pattern = r'\b(' + joined_contractions + r')\b'
compiled_pattern = re.compile(contractions_pattern, flags=re.IGNORECASE)

def replaced_contractions(text):
    def replace_match(match):
        matched_word = match.group(0)  # Extract matched contraction
        lower_matched_word = matched_word.lower()  # Convert to lowercase
        expanded_form = contractions_dict[lower_matched_word]  # Get full form from dictionary
        return expanded_form  # Return the expanded form

    expanded_text = compiled_pattern.sub(replace_match, text)

    return expanded_text

df['contractions_replaced'] = df['slangs_replaced'].apply(replaced_contractions)

In [14]:
# Step 4: Remove punctuation from text
import string
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df["punctuations_removed"] = df["contractions_replaced"].apply(remove_punctuation)

# Step 5: Remove numbers form the text
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

df['numbers_removed'] = df['punctuations_removed'].apply(remove_numbers)

In [15]:
# Step 6: Remove emojis from the text
import emoji
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

df['emoji_removed'] = df['punctuations_removed'].apply(remove_emojis)

# Step 7: Remove stopwords from the text
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = []
    for word in words:
        lower_word = word.lower()
        if lower_word not in stop_words:
            filtered_words.append(word)
    return " ".join(filtered_words)

df["stopwords_removed"] = df["emoji_removed"].apply(remove_stopwords)

# Step 8: Stem all the text
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def stem_text(text):
    if not isinstance(text, str):
        return ""
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

df["stemmed_words"] = df["stopwords_removed"].apply(stem_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thema\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Step 9: Lemmatize the text
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_text(text):
    if not isinstance(text, str):
        return ""

    words = word_tokenize(text)
    pos_tags = pos_tag(words)

    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return " ".join(lemmatized_words)

df["lemmatized"] = df["stopwords_removed"].apply(lemmatize_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thema\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\thema\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\thema\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\thema\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [18]:
# Step 10: Tokenize all the text
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenize_text(text):
    if not isinstance(text, str):
        return []
    return word_tokenize(text)

df["tokenized"] = df["lemmatized"].apply(tokenize_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thema\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
df

,Unnamed: 0,text,target,title,date,lowercased,slangs_replaced,contractions_replaced,punctuations_removed,numbers_removed,emoji_removed,stopwords_removed,stemmed_words,lemmatized,tokenized
0,0,I was wondering if anyone out there could enli...,7,rec.autos,2022-08-02 13:48:37.251043,i was wondering if anyone out there could enli...,i was wondering if anyone out there could enli...,i was wondering if anyone out there could enli...,i was wondering if anyone out there could enli...,i was wondering if anyone out there could enli...,i was wondering if anyone out there could enli...,wondering anyone could enlighten car saw day 2...,wonder anyon could enlighten car saw day 2door...,wonder anyone could enlighten car saw day 2doo...,"[wonder, anyone, could, enlighten, car, saw, d..."
1,17,I recently posted an article asking what kind ...,7,rec.autos,2022-08-02 13:48:37.251043,i recently posted an article asking what kind ...,i recently posted an article asking what kind ...,i recently posted an article asking what kind ...,i recently posted an article asking what kind ...,i recently posted an article asking what kind ...,i recently posted an article asking what kind ...,recently posted article asking kind rates sing...,recent post articl ask kind rate singl male dr...,recently post article ask kind rate single mal...,"[recently, post, article, ask, kind, rate, sin..."
2,29,\nIt depends on your priorities. A lot of peo...,7,rec.autos,2022-08-02 13:48:37.251043,\nit depends on your priorities. a lot of peo...,\nit depends on your priorities. a lot of peo...,\nit depends on your priorities. a lot of peo...,\nit depends on your priorities a lot of peop...,\nit depends on your priorities a lot of peop...,\nit depends on your priorities a lot of peop...,depends priorities lot people put higher prior...,depend prioriti lot peopl put higher prioriti ...,depend priority lot people put high priority g...,"[depend, priority, lot, people, put, high, pri..."
3,56,an excellent automatic can be found in the sub...,7,rec.autos,2022-08-02 13:48:37.251043,an excellent automatic can be found in the sub...,an excellent automatic can be found in the sub...,an excellent automatic can be found in the sub...,an excellent automatic can be found in the sub...,an excellent automatic can be found in the sub...,an excellent automatic can be found in the sub...,excellent automatic found subaru legacy switch...,excel automat found subaru legaci switch sport...,excellent automatic find subaru legacy switch ...,"[excellent, automatic, find, subaru, legacy, s..."
4,64,: Ford and his automobile. I need information...,7,rec.autos,2022-08-02 13:48:37.251043,: ford and his automobile. i need information...,: ford and his automobile. i need information...,: ford and his automobile. i need information...,ford and his automobile i need information o...,ford and his automobile i need information o...,ford and his automobile i need information o...,ford automobile need information whether ford ...,ford automobil need inform whether ford partia...,ford automobile need information whether ford ...,"[ford, automobile, need, information, whether,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11309,11210,Secrecy in Clipper Chip\n\nThe serial number o...,11,sci.crypt,2022-08-02 13:48:37.251043,secrecy in clipper chip\n\nthe serial number o...,secrecy in clipper chip\n\nthe serial number o...,secrecy in clipper chip\n\nthe serial number o...,secrecy in clipper chip\n\nthe serial number o...,secrecy in clipper chip\n\nthe serial number o...,secrecy in clipper chip\n\nthe serial number o...,secrecy clipper chip serial number clipper chi...,secreci clipper chip serial number clipper chi...,secrecy clipper chip serial number clipper chi...,"[secrecy, clipper, chip, serial, number, clipp..."
11310,11217,Hi !\n\nI am interested in the source of FEAL ...,11,sci.crypt,2022-08-02 13:48:37.251043,hi !\n\ni am interested in the source of feal ...,hi !\n\ni am interested in the

In [26]:
# Save to the cleaned dataset to a CSV file
df[["text", "tokenized"]].to_csv("cleaned_news_data.csv")

# Topic Modeling

In [30]:
df.head()

,Unnamed: 0,text,target,title,date,lowercased,slangs_replaced,contractions_replaced,punctuations_removed,numbers_removed,emoji_removed,stopwords_removed,stemmed_words,lemmatized,tokenized
0,0,I was wondering if anyone out there could enli...,7,rec.autos,2022-08-02 13:48:37.251043,i was wondering if anyone out there could enli...,i was wondering if anyone out there could enli...,i was wondering if anyone out there could enli...,i was wondering if anyone out there could enli...,i was wondering if anyone out there could enli...,i was wondering if anyone out there could enli...,wondering anyone could enlighten car saw day 2...,wonder anyon could enlighten car saw day 2door...,wonder anyone could enlighten car saw day 2doo...,"[wonder, anyone, could, enlighten, car, saw, d..."
1,17,I recently posted an article asking what kind ...,7,rec.autos,2022-08-02 13:48:37.251043,i recently posted an article asking what kind ...,i recently posted an article asking what kind ...,i recently posted an article asking what kind ...,i recently posted an article asking what kind ...,i recently posted an article asking what kind ...,i recently posted an article asking what kind ...,recently posted article asking kind rates sing...,recent post articl ask kind rate singl male dr...,recently post article ask kind rate single mal...,"[recently, post, article, ask, kind, rate, sin..."
2,29,\nIt depends on your priorities. A lot of peo...,7,rec.autos,2022-08-02 13:48:37.251043,\nit depends on your priorities. a lot of peo...,\nit depends on your priorities. a lot of peo...,\nit depends on your priorities. a lot of peo...,\nit depends on your priorities a lot of peop...,\nit depends on your priorities a lot of peop...,\nit depends on your priorities a lot of peop...,depends priorities lot people put higher prior...,depend prioriti lot peopl put higher prioriti ...,depend priority lot people put high priority g...,"[depend, priority, lot, people, put, high, pri..."
3,56,an excellent automatic can be found in the sub...,7,rec.autos,2022-08-02 13:48:37.251043,an excellent automatic can be found in the sub...,an excellent automatic can be found in the sub...,an excellent automatic can be found in the sub...,an excellent automatic can be found in the sub...,an excellent automatic can be found in the sub...,an excellent automatic can be found in the sub...,excellent automatic found subaru legacy switch...,excel automat found subaru legaci switch sport...,excellent automatic find subaru legacy switch ...,"[excellent, automatic, find, subaru, legacy, s..."
4,64,: Ford and his automobile. I need information...,7,rec.autos,2022-08-02 13:48:37.251043,: ford and his automobile. i need information...,: ford and his automobile. i need information...,: ford and his automobile. i need information...,ford and his automobile i need information o...,ford and his automobile i need information o...,ford and his automobile i need information o...,ford automobile need information whether ford ...,ford automobil need inform whether ford partia...,ford automobile need information whether ford ...,"[ford, automobile, need, information, whether,..."


In [32]:
# Create document-term matrix
dictionary = corpora.Dictionary(df["tokenized"])
dictionary.filter_extremes(no_below=15, no_above=0.5)
corpus = [dictionary.doc2bow(doc) for doc in df["tokenized"]]

In [34]:
# Train an LDA model on the corpus with 2 topics using Gensim's LdaModel class
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [47]:
# Interpret results
documents = df['text'].tolist()
article_labels = []
for i, doc in enumerate(df["tokenized"]):
    bow = dictionary.doc2bow(doc)
    topics = lda_model.get_document_topics(bow)
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    article_labels.append(dominant_topic)
    
df_result = pd.DataFrame({"Article": documents, "Topic": article_labels})

# Print the DataFrame
print("Table with Articles and Topic:")
print(df_result)
print()

Table with Articles and Topic:
                                                 Article  Topic
0      I was wondering if anyone out there could enli...      2
1      I recently posted an article asking what kind ...      2
2      \nIt depends on your priorities.  A lot of peo...      2
3      an excellent automatic can be found in the sub...      2
4      : Ford and his automobile.  I need information...      2
...                                                  ...    ...
11309  Secrecy in Clipper Chip\n\nThe serial number o...      0
11310  Hi !\n\nI am interested in the source of FEAL ...      0
11311  The actual algorithm is classified, however, t...      0
11312  \n\tThis appears to be generic calling upon th...      1
11313  \nProbably keep quiet and take it, lest they g...      1

[11314 rows x 2 columns]



In [38]:
# Print top terms for each topic
for topic_id in range(lda_model.num_topics):
    print(f"Top terms for Topic #{topic_id}:")
    top_terms = lda_model.show_topic(topic_id, topn=10)
    print([term[0] for term in top_terms])
    print()

Top terms for Topic #0:
['use', 'key', 'system', 'information', 'db', 'encryption', 'chip', 'data', 'available', 'one']

Top terms for Topic #1:
['would', 'say', 'one', 'think', 'people', 'go', 'know', 'make', 'get', 'see']

Top terms for Topic #2:
['get', 'use', 'would', 'one', 'like', 'know', 'work', 'problem', 'drive', 'good']

Top terms for Topic #3:
['government', 'state', 'q', 'armenian', 'people', 'president', 'mr', 'u', 'say', 'right']

Top terms for Topic #4:
['x', '1', '2', '0', '3', '4', 'file', '5', '6', '7']



In [40]:
# Print the top terms for each topic with weight
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
    print()

Top Terms for Each Topic:
Topic 0:
- "use" (weight: 0.017)
- "key" (weight: 0.016)
- "system" (weight: 0.010)
- "information" (weight: 0.008)
- "db" (weight: 0.008)
- "encryption" (weight: 0.007)
- "chip" (weight: 0.007)
- "data" (weight: 0.006)
- "available" (weight: 0.005)
- "one" (weight: 0.005)

Topic 1:
- "would" (weight: 0.014)
- "say" (weight: 0.012)
- "one" (weight: 0.012)
- "think" (weight: 0.010)
- "people" (weight: 0.009)
- "go" (weight: 0.009)
- "know" (weight: 0.008)
- "make" (weight: 0.008)
- "get" (weight: 0.007)
- "see" (weight: 0.006)

Topic 2:
- "get" (weight: 0.014)
- "use" (weight: 0.013)
- "would" (weight: 0.013)
- "one" (weight: 0.010)
- "like" (weight: 0.009)
- "know" (weight: 0.008)
- "work" (weight: 0.008)
- "problem" (weight: 0.007)
- "drive" (weight: 0.006)
- "good" (weight: 0.006)

Topic 3:
- "government" (weight: 0.009)
- "state" (weight: 0.008)
- "q" (weight: 0.008)
- "armenian" (weight: 0.008)
- "people" (weight: 0.008)
- "president" (weight: 0.007)
- "mr

**Student Name**: Alasiri Omar Abdullah

**Student ID**: IS01082222

**Section 03**

**Result Interpretation**

**Topic 0** seems to be about Cybersecurity / Information Systems theme, we can see the term "use" and "key" that may indicate that people using information systems, we also see the two terms "system" and "information" with "encription" and "available" terms can indicate the wated secyrity for these technologies. The terms clearly align with encryption, databases, and secure information systems. Likely a high coherence score.

**Topic 1** seems to be related to General Conversation / Dialogue theme, or people want to know and invistigate a specific event they experienced, we can see the terms "people", "go", "know", "see", "get" are frequently mentioned, we can also see the terms "would" and "say" that can indicate that news has spread quickly, meaning that people say things that would happen. Generic verbs and pronouns, possibly too abstract or broad. Likely a low coherence score due to lack of clear thematic connection.

**Topic 2** seems to be about General Actions / Common Speech, or getting to work and the problem of driving because of a traffic jam that is frequently occur because many people trying to get to work at the same time causing the traffic jam. We can see the terms "get", "use", "work", "drive", and "problem" that can indicate that. Similar to Topic 1; includes many functional or frequently used words. May indicate noise or overlapping topics. Low coherence.

**Topic 3** looks like about Armenia, which is a small country that share boarders with Turkey and Iran, it seems like a Politics / Geopolitics (possibly Armenian context) theme, Armania is a republic, we can see terms "armenian", "governemnt", "state", "people", "president", and "right", these terms can indicate that the people of Armenia is supporting their government, the terms "president", "mr", "say", and "right" can indicate that the armenian people think that their president is saying the right things. Could be strong with more context. Terms suggest a political discussion, but some generic (e.g., "u", "say", "right") terms dilute clarity. Coherence might be moderate.